# LSTM
Using LSTM to predict price

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import proprocessing
import matplotlib.pyplot as plt

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: cannot import name 'proprocessing'

## Process the train and test data

In [2]:
def readTrain(filename = 'train_data.csv'):
	df = pd.read_csv(filename)
	train_data = df.iloc[:,3:10]
	return train_data

In [3]:
def normalize(train):
	train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
	return train_norm

In [4]:
def buildTrain(train, pastDay=10, futureDay=20):
	train = np.array(train)
	X_train, Y_train = [], []
	for i in range(train.shape[0]-futureDay-pastDay):
		X_train.append(train.iloc[i:i+pastDay])
		Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["MidPrice"]).mean())
	return np.array(X_train), np.array(Y_train)

In [5]:
def shuffle(X,Y):
	np.random.seed(10)
	randomList = np.arange(X.shape[0])
	np.random.shuffle(randomList)
	return X[randomList], Y[randomList]

In [6]:
def splitData(X,Y,rate):
	X_train = X[int(X.shape[0]*rate):]
	Y_train = Y[int(Y.shape[0]*rate):]
	X_val = X[:int(X.shape[0]*rate)]
	Y_val = Y[:int(Y.shape[0]*rate)]
	return X_train, Y_train, X_val, Y_val

In [7]:
train = readTrain()


,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1
0,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0
1,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0
2,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0
3,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0
4,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0
5,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0
6,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0
7,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0
8,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0
9,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0


In [8]:
train_norm = normalize(train)
train_norm

,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1
0,0.489980,0.489125,-0.217164,0.490124,-0.007365,0.489835,-0.024743
1,0.485078,0.485857,-0.216442,0.485222,-0.006631,0.484933,-0.013005
2,0.485078,0.485857,-0.216317,0.485222,-0.006680,0.484933,-0.019421
3,0.486712,0.487491,-0.216178,0.486856,-0.003343,0.486567,-0.024302
4,0.485078,0.485857,-0.215106,0.485222,0.003183,0.484933,-0.004072
5,0.485078,0.485857,-0.214382,0.485222,0.003341,0.484933,-0.018361
6,0.489163,0.487491,-0.214202,0.486856,0.005359,0.491469,-0.025527
7,0.489163,0.487491,-0.214202,0.486856,-0.003753,0.491469,-0.016682
8,0.489163,0.485857,-0.213957,0.486856,-0.007527,0.491469,-0.016682
9,0.488346,0.485857,-0.213941,0.486856,-0.007776,0.489835,-0.017323


In [11]:
def buildTrain(train, pastDay=10, futureDay=20):
	train = train.values
	X_train, Y_train = [], []
	for i in range(train.shape[0]-futureDay-pastDay):
		X_train.append(train[i:i+pastDay,:])
		Y_train.append(train[i+pastDay:i+pastDay+futureDay,0].mean())
	return np.array(X_train), np.array(Y_train)
# build Data, use last 10 days to predict next 20 days
X_train, Y_train = buildTrain(train_norm, 10, 20)
print(X_train.shape)
print(Y_train.shape)

(430009, 10, 7)
(430009,)


In [12]:
# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [36]:
def buildManyToOneModel(shape):
	model = Sequential()
	model.add(LSTM(128, input_length=shape[1], input_dim=shape[2],return_sequences=True))
	model.add(LSTM(128,return_sequences = False))
	# output shape: (1, 1)
	model.add(Dense(1))
	model.compile(loss="mse", optimizer="adam")
	model.summary()
	return model

In [37]:
model = buildManyToOneModel(X_train.shape)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, return_sequences=True, input_shape=(10, 7))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 128)           69632     
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 201,345
Trainable params: 201,345
Non-trainable params: 0
_________________________________________________________________


In [42]:
model = load_model('double_LSTM_v2.h5')
callback = EarlyStopping(monitor="loss", patience=3, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=5, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])
model.save('double_LSTM_v2.h5')

Train on 387009 samples, validate on 43000 samples
Epoch 1/5
387009/387009 [==============================] - 206s 532us/step - loss: 5.9074e-06 - val_loss: 5.3673e-06
Epoch 2/5
387009/387009 [==============================] - 199s 515us/step - loss: 5.9298e-06 - val_loss: 5.6380e-06
Epoch 3/5
387009/387009 [==============================] - 198s 513us/step - loss: 5.9143e-06 - val_loss: 6.0054e-06
Epoch 4/5
387009/387009 [==============================] - 200s 516us/step - loss: 5.8883e-06 - val_loss: 5.5776e-06
Epoch 5/5
387009/387009 [==============================] - 201s 519us/step - loss: 5.8834e-06 - val_loss: 5.3055e-06


In [16]:
from keras.models import load_model
def load_test_data():
	f = open('test_data.csv')
	df = pd.read_csv(f)
	test_data = df.iloc[:,3:10].values
	test_datas = []
	i = 0
	while(i<len(test_data)):
		test_datas.append(test_data[i:i+10,:])
		i = i + 10
	test_datas = np.array(test_datas)
	return test_datas

In [40]:
model = load_model('double_LSTM_v2.h5')
test_X_pd = readTrain('test_data.csv').iloc[1420:,:]
test_X = normalize(test_X_pd)
test_X = np.reshape(test_X.values,(-1,10,7))
test_tmp_x_mean = np.mean(test_X_pd)
test_tmp_x_max_min = np.max(test_X_pd)-np.min(test_X_pd)
prediction = model.predict(test_X)
prediction = prediction * test_tmp_x_max_min['MidPrice'] + test_tmp_x_mean['MidPrice']
prediction = np.reshape(prediction,(-1))

In [41]:
caseid = list(range(143,1001))
df = pd.DataFrame({'caseid':caseid, 'midprice':prediction})
df.to_csv('doubleLSTM_v2.csv',index = False, sep = ',')

In [ ]:
test_X_pd = readTrain('test_data.csv').iloc[1420:,:]
prediction  = np.zeros(())
test_X = normalize(test_X_pd)
test_X = np.reshape(test_X.values,(-1,10,7))
test_tmp_x_mean = np.mean(test_X_pd)
test_tmp_x_max_min = np.max(test_X_pd)-np.min(test_X_pd)
prediction = model.predict(test_X)
prediction = prediction * test_tmp_x_max_min['MidPrice'] + test_tmp_x_mean['MidPrice']
prediction = np.reshape(prediction,(-1))